In [1]:
import pandas as pd
import numpy as np
import os
import random

from mlchartist.array_builder import build_arrays, build_randomised_arrays
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

In [2]:
df = pd.read_csv('../raw_data/processed/aapl.csv')

In [3]:
df['date'] = pd.to_datetime(df['date'])

In [4]:
FIVE_TR = 0.0006
TEN_TR = 0.0012
TWENTY_TR = 0.0024
INPUT_COLS = ['RSI', 'Stochastic', 'Stochastic_signal', 'ADI','OBV', 'ATR', 'ADX', 'ADX_pos', 'ADX_neg', 'MACD', 'MACD_diff','MACD_signal', '1D_past_return', '5D_past_return', '10D_past_return']
#INPUT_COLS = ['RSI', 'Stochastic', 'Stochastic_signal']

In [5]:
df['5D_return_bin'] = (df['5TD_return'] >= FIVE_TR)
df['10D_return_bin'] = (df['10TD_return'] >= TEN_TR)
df['20D_return_bin'] = (df['20TD_return'] >= TWENTY_TR)

In [6]:
test_df = df[(df['date'].dt.year >= 2017) & (df['date'].dt.year <= 2019)]
train_df = df[(df['date'].dt.year >= 1995) & (df['date'].dt.year < 2017)]

In [7]:
scaler = StandardScaler()

In [8]:
scaler.fit(train_df[INPUT_COLS])

StandardScaler()

In [9]:
train_df.loc[:, INPUT_COLS] = scaler.transform(train_df[INPUT_COLS])
test_df.loc[:, INPUT_COLS] = scaler.transform(test_df[INPUT_COLS])

/home/kensei/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
/home/kensei/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [10]:
train_df

,ticker,date,RSI,Stochastic,Stochastic_signal,ADI,OBV,ATR,ADX,ADX_pos,...,MACD_signal,5TD_return,10TD_return,20TD_return,1D_past_return,5D_past_return,10D_past_return,5D_return_bin,10D_return_bin,20D_return_bin
2576,AAPL,1995-01-03,-0.276126,0.130640,0.760803,-0.973950,-1.750625,-0.787427,-1.116899,0.127035,...,-0.177532,0.137530,0.171710,0.051861,-0.564551,-0.284328,0.207511,True,True,True
2577,AAPL,1995-01-04,0.206404,0.987085,0.678144,-0.971123,-1.747888,-0.787424,-1.142106,0.414668,...,-0.175352,0.186767,0.157210,0.018156,0.853992,0.014979,-0.058760,True,True,True
2578,AAPL,1995-01-05,-0.083517,0.394701,0.538988,-0.972516,-1.749158,-0.789080,-1.165512,0.282975,...,-0.173616,0.168014,0.180473,0.070379,-0.509115,-0.197944,-0.028111,True,True,True
2579,AAPL,1995-01-06,1.011928,0.873063,0.803622,-0.976448,-1.730592,-0.779607,-0.982966,1.817740,...,-0.170235,0.068410,0.014344,-0.036508,2.735552,0.883627,0.894121,True,True,False
2580,AAPL,1995-01-09,0.623690,0.387398,0.589865,-0.980846,-1.735318,-0.780756,-0.824343,1.570206,...,-0.166525,0.079660,0.025272,-0.017867,-0.697859,0.790160,0.601116,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8108,AAPL,2016-12-23,0.916686,1.081715,1.219224,1.336925,1.107500,1.259392,-0.214894,-0.034720,...,1.225535,-0.003226,0.022144,0.045883,0.025916,-0.022024,0.112079,False,True,True
8109,AAPL,2016-12-27,1.110562,1.244353,1.198237,1.337231,1.108599,1.230710,-0.191890,0.456505,...,1.334865,-0.010410,0.021288,0.039983,0.169578,-0.017361,0.245686,False,True,True
8110,AAPL,2016-12-28,0.848563,0.899185,1.149520,1.336436,1.107386,1.252774,-0.197010,0.186792,...,1.416748,-0.001194,0.021413,0.044417,-0.185915,-0.122105,0.014513,False,True,True
8111,AAPL,2016-12-29,0.841578,0.727793,1.023366,1.336398,1.106459,1.169928,-0.201763,0.085747,...,1.473255,0.009876,0.019606,0.041637,-0.046739,-0.137891,0.013302,True,True,True


In [11]:
apple_train_x_20, apple_train_y_20 = build_arrays(train_df,input_cols=INPUT_COLS, target_col='20D_return_bin', time_window=40, stride=1)
apple_test_x_20, apple_test_y_20 = build_arrays(test_df,input_cols=INPUT_COLS, target_col='20D_return_bin', time_window=40, stride=1)

In [12]:
indx = list(range(len(apple_train_x_20)))

In [13]:
len(indx)

5498

In [14]:
test_df.shape

(754, 23)

In [15]:
sample_indx = random.sample(indx, 5000)

In [16]:
X_train_20 =  apple_train_x_20[[sample_indx], :][0]

In [17]:
y_train_20 = apple_train_y_20[[sample_indx]]

/home/kensei/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """Entry point for launching an IPython kernel.


In [18]:
from tensorflow.keras.metrics import Precision
from tensorflow.keras import regularizers

In [19]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers, models 
from tensorflow.keras.optimizers import RMSprop, Adam

optim = RMSprop(learning_rate=0.0001)
precision = Precision()

def init_model():
    model = Sequential()
    reg_l1 = regularizers.l1(0.001)
    reg_l2 = regularizers.l2(0.001)
    reg_l1_l2 = regularizers.l1_l2(l1=0.001, l2=0.001)
    model.add(layers.LSTM(200, return_sequences=True, input_shape=(40,15), activation='tanh'))
    model.add(layers.LSTM(200, activation='tanh'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(200, activation='relu', kernel_regularizer=reg_l1))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(100, activation='relu', bias_regularizer=reg_l2))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(50, activation='relu', activity_regularizer=reg_l1_l2))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=optim, metrics=[precision, 'accuracy'])
    
    return model

### Model for 20 days future returns

In [22]:
model_20 = init_model()

from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=3, restore_best_weights=True)

model_20.fit(X_train_20, y_train_20, 
          epochs=500, 
          batch_size=16,
          validation_split=0.2,
          callbacks=[es])

Epoch 1/500
250/250 [==============================] - 2s 9ms/step - loss: 2.7492 - precision: 0.6261 - accuracy: 0.5748 - val_loss: 2.4251 - val_precision: 0.6182 - val_accuracy: 0.6130
Epoch 2/500
250/250 [==============================] - 2s 7ms/step - loss: 2.1587 - precision: 0.6048 - accuracy: 0.6025 - val_loss: 1.9205 - val_precision: 0.6374 - val_accuracy: 0.6200
Epoch 3/500
250/250 [==============================] - 2s 9ms/step - loss: 1.7159 - precision: 0.6415 - accuracy: 0.6263 - val_loss: 1.5365 - val_precision: 0.6955 - val_accuracy: 0.6670
Epoch 4/500
250/250 [==============================] - 1s 6ms/step - loss: 1.3912 - precision: 0.6809 - accuracy: 0.6540 - val_loss: 1.2922 - val_precision: 0.6656 - val_accuracy: 0.6330
Epoch 5/500
250/250 [==============================] - 2s 6ms/step - loss: 1.1556 - precision: 0.6941 - accuracy: 0.6680 - val_loss: 1.0673 - val_precision: 0.6840 - val_accuracy: 0.6500
Epoch 6/500
250/250 [==============================] - 2s 7ms/ste

In [23]:
apple_train_y_20.sum()/len(apple_train_y_20)

0.5725718443070207

In [24]:
apple_test_y_20.sum()/len(apple_test_y_20)

0.6853146853146853

In [25]:
model_20.evaluate(apple_test_x_20, apple_test_y_20)

23/23 [==============================] - 0s 3ms/step - loss: 1.3471 - precision: 0.7056 - accuracy: 0.5217


[1.3471311330795288, 0.7055555582046509, 0.5216783285140991]

### Model for 10 days future returns

In [26]:
apple_train_x_10, apple_train_y_10 = build_arrays(train_df,input_cols=INPUT_COLS, target_col='10D_return_bin', time_window=40, stride=1)
apple_test_x_10, apple_test_y_10 = build_arrays(test_df,input_cols=INPUT_COLS, target_col='10D_return_bin', time_window=40, stride=1)

In [27]:
X_train_10 =  apple_train_x_10[[sample_indx], :][0]
y_train_10 = apple_train_y_10[[sample_indx]]

/home/kensei/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


In [28]:
model_10 = init_model()

from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=3, restore_best_weights=True)

model_10.fit(X_train_10, y_train_10, 
          epochs=500, 
          batch_size=16,
          validation_split=0.2,
          callbacks=[es])

Epoch 1/500
250/250 [==============================] - 2s 8ms/step - loss: 2.7798 - precision: 0.5850 - accuracy: 0.5353 - val_loss: 2.4436 - val_precision: 0.5895 - val_accuracy: 0.6000
Epoch 2/500
250/250 [==============================] - 1s 6ms/step - loss: 2.1739 - precision: 0.5843 - accuracy: 0.5842 - val_loss: 1.9221 - val_precision: 0.6066 - val_accuracy: 0.5950
Epoch 3/500
250/250 [==============================] - 1s 6ms/step - loss: 1.7233 - precision: 0.5906 - accuracy: 0.5855 - val_loss: 1.5379 - val_precision: 0.6279 - val_accuracy: 0.6040
Epoch 4/500
250/250 [==============================] - 2s 6ms/step - loss: 1.3917 - precision: 0.6082 - accuracy: 0.5995 - val_loss: 1.2577 - val_precision: 0.6282 - val_accuracy: 0.6210
Epoch 5/500
250/250 [==============================] - 2s 7ms/step - loss: 1.1550 - precision: 0.6167 - accuracy: 0.6137 - val_loss: 1.0612 - val_precision: 0.6361 - val_accuracy: 0.6200
Epoch 6/500
250/250 [==============================] - 1s 6ms/ste

In [29]:
apple_test_y_10.sum()/len(apple_test_y_10)

0.641958041958042

In [30]:
model_10.evaluate(apple_test_x_10, apple_test_y_10)

23/23 [==============================] - 0s 3ms/step - loss: 0.9718 - precision: 0.6716 - accuracy: 0.5846


[0.9718372821807861, 0.6716101765632629, 0.5846154093742371]

### Model for 5 days future returns

In [32]:
apple_train_x_5, apple_train_y_5 = build_arrays(train_df,input_cols=INPUT_COLS, target_col='5D_return_bin', time_window=40, stride=1)
apple_test_x_5, apple_test_y_5 = build_arrays(test_df,input_cols=INPUT_COLS, target_col='5D_return_bin', time_window=40, stride=1)

In [33]:
X_train_5 =  apple_train_x_5[[sample_indx], :][0]
y_train_5 = apple_train_y_5[[sample_indx]]

/home/kensei/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


In [34]:
model_5 = init_model()

from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=3, restore_best_weights=True)

model_5.fit(X_train_5, y_train_5, 
          epochs=500, 
          batch_size=16,
          validation_split=0.2,
          callbacks=[es])

Epoch 1/500
250/250 [==============================] - 2s 10ms/step - loss: 2.7442 - precision: 0.5604 - accuracy: 0.5322 - val_loss: 2.3977 - val_precision: 0.5335 - val_accuracy: 0.5380
Epoch 2/500
250/250 [==============================] - 2s 7ms/step - loss: 2.1086 - precision: 0.5546 - accuracy: 0.5543 - val_loss: 1.8472 - val_precision: 0.5552 - val_accuracy: 0.5680
Epoch 3/500
250/250 [==============================] - 2s 9ms/step - loss: 1.6319 - precision: 0.5630 - accuracy: 0.5620 - val_loss: 1.4419 - val_precision: 0.5654 - val_accuracy: 0.5700
Epoch 4/500
250/250 [==============================] - 2s 8ms/step - loss: 1.2819 - precision: 0.5783 - accuracy: 0.5730 - val_loss: 1.1451 - val_precision: 0.5769 - val_accuracy: 0.5780
Epoch 5/500
250/250 [==============================] - 2s 8ms/step - loss: 1.0393 - precision: 0.5957 - accuracy: 0.5835 - val_loss: 0.9486 - val_precision: 0.5898 - val_accuracy: 0.5800
Epoch 6/500
250/250 [==============================] - 2s 8ms/st

In [35]:
apple_test_y_5.sum()/len(apple_test_y_5)

0.6111888111888112

In [36]:
model_5.evaluate(apple_test_x_5, apple_test_y_5)

23/23 [==============================] - 0s 3ms/step - loss: 0.7412 - precision: 0.6319 - accuracy: 0.5483


[0.7412089705467224, 0.6319444179534912, 0.548251748085022]

# CNN test

### CNN for 20 days future returns

In [288]:
from tensorflow.keras.backend import expand_dims
X_train_cnn_20 = expand_dims(X_train_20, axis=-1)
X_test_cnn_20 = expand_dims(apple_test_x_20, axis=-1)


In [289]:
def initialize_model_cnn():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), padding='same', input_shape=(40, 15, 1)))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(64, (3, 3), padding='same'))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(128, (3, 3), padding='same'))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(120, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(60, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [290]:
model_cnn_20 = initialize_model_cnn()

model_cnn_20.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

es = EarlyStopping(patience=5, restore_best_weights=True)

model_cnn_20.fit(X_train_cnn_20, y_train_20, 
          epochs=500, 
          batch_size=16,
          validation_split=0.2,
            callbacks=[es])

Epoch 1/500
250/250 [==============================] - 1s 3ms/step - loss: 0.6852 - accuracy: 0.5670 - val_loss: 0.6656 - val_accuracy: 0.5830
Epoch 2/500
250/250 [==============================] - 0s 2ms/step - loss: 0.6724 - accuracy: 0.5767 - val_loss: 0.6574 - val_accuracy: 0.5900
Epoch 3/500
250/250 [==============================] - 1s 3ms/step - loss: 0.6487 - accuracy: 0.6097 - val_loss: 0.6172 - val_accuracy: 0.6400
Epoch 4/500
250/250 [==============================] - 1s 3ms/step - loss: 0.6152 - accuracy: 0.6495 - val_loss: 0.5857 - val_accuracy: 0.6950
Epoch 5/500
250/250 [==============================] - 1s 3ms/step - loss: 0.5692 - accuracy: 0.6965 - val_loss: 0.5227 - val_accuracy: 0.7250
Epoch 6/500
250/250 [==============================] - 1s 2ms/step - loss: 0.4995 - accuracy: 0.7655 - val_loss: 0.5045 - val_accuracy: 0.7340
Epoch 7/500
250/250 [==============================] - 1s 2ms/step - loss: 0.4520 - accuracy: 0.7950 - val_loss: 0.4304 - val_accuracy: 0.8150

In [291]:
model_cnn_20.evaluate(X_test_cnn_20, apple_test_y_20)

23/23 [==============================] - 0s 2ms/step - loss: 5.2828 - accuracy: 0.4364 


[5.282845497131348, 0.4363636374473572]

### CNN for 10 days future returns

In [292]:
X_train_cnn_10 = expand_dims(X_train_10, axis=-1)
X_test_cnn_10 = expand_dims(apple_test_x_10, axis=-1)

In [ ]:
model_cnn_10 = initialize_model_cnn()

model_cnn_10.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

es = EarlyStopping(patience=5, restore_best_weights=True)

model_cnn_10.fit(X_train_cnn_10, y_train_10, 
          epochs=500, 
          batch_size=16,
          validation_split=0.2,
            callbacks=[es])

Epoch 1/500
250/250 [==============================] - 1s 3ms/step - loss: 0.6943 - accuracy: 0.5315 - val_loss: 0.6884 - val_accuracy: 0.5670
Epoch 2/500
250/250 [==============================] - 1s 3ms/step - loss: 0.6887 - accuracy: 0.5540 - val_loss: 0.6843 - val_accuracy: 0.5640
Epoch 3/500
250/250 [==============================] - 1s 2ms/step - loss: 0.6840 - accuracy: 0.5497 - val_loss: 0.6807 - val_accuracy: 0.5780
Epoch 4/500
250/250 [==============================] - 1s 2ms/step - loss: 0.6793 - accuracy: 0.5675 - val_loss: 0.6742 - val_accuracy: 0.5850
Epoch 5/500
250/250 [==============================] - 1s 3ms/step - loss: 0.6687 - accuracy: 0.5817 - val_loss: 0.6578 - val_accuracy: 0.6150
Epoch 6/500
250/250 [==============================] - 1s 2ms/step - loss: 0.6559 - accuracy: 0.6202 - val_loss: 0.6360 - val_accuracy: 0.6080
Epoch 7/500
250/250 [==============================] - 1s 2ms/step - loss: 0.6284 - accuracy: 0.6388 - val_loss: 0.6216 - val_accuracy: 0.6560

In [254]:
model_cnn_10.evaluate(X_test_cnn_10, apple_test_y_10)

22/22 [==============================] - 0s 1ms/step - loss: 2.3879 - accuracy: 0.5065


[2.38793683052063, 0.5064747929573059]

### CNN for 5 days future returns

In [255]:
X_train_cnn_5 = expand_dims(X_train_5, axis=-1)
X_test_cnn_5 = expand_dims(apple_test_x_5, axis=-1)

In [256]:
model_cnn_5 = initialize_model_cnn()

model_cnn_5.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

es = EarlyStopping(patience=5, restore_best_weights=True)

model_cnn_5.fit(X_train_cnn_5, y_train_5, 
          epochs=500, 
          batch_size=32,
          validation_split=0.2,
            callbacks=[es])

Epoch 1/500
125/125 [==============================] - 0s 3ms/step - loss: 0.6949 - accuracy: 0.5217 - val_loss: 0.6860 - val_accuracy: 0.5590
Epoch 2/500
125/125 [==============================] - 0s 2ms/step - loss: 0.6914 - accuracy: 0.5315 - val_loss: 0.6900 - val_accuracy: 0.5590
Epoch 3/500
125/125 [==============================] - 0s 3ms/step - loss: 0.6893 - accuracy: 0.5430 - val_loss: 0.6845 - val_accuracy: 0.5660
Epoch 4/500
125/125 [==============================] - 0s 2ms/step - loss: 0.6900 - accuracy: 0.5397 - val_loss: 0.6930 - val_accuracy: 0.5280
Epoch 5/500
125/125 [==============================] - 0s 3ms/step - loss: 0.6884 - accuracy: 0.5433 - val_loss: 0.6833 - val_accuracy: 0.5600
Epoch 6/500
125/125 [==============================] - 0s 3ms/step - loss: 0.6868 - accuracy: 0.5355 - val_loss: 0.6809 - val_accuracy: 0.5640
Epoch 7/500
125/125 [==============================] - 0s 2ms/step - loss: 0.6820 - accuracy: 0.5562 - val_loss: 0.6740 - val_accuracy: 0.5990

In [257]:
model_cnn_5.evaluate(X_test_cnn_5, apple_test_y_5)

22/22 [==============================] - 0s 1ms/step - loss: 1.0793 - accuracy: 0.4662


[1.0792920589447021, 0.466187059879303]